In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [38]:
csv = pd.read_csv(r'../data/tripadvisor_hotel_reviews.csv')

In [39]:
sentiment_dict = {1: "Negative",
                 2: "Neutral",
                 3: "Positive"}

In [40]:
def rating_to_sentiment(rating):
    if rating>3 and rating<=5:
        return 3
    elif rating == 3:
        return 2
    else:
        return 1

In [41]:
tqdm.pandas()
csv['Sentiment'] = csv['Rating'].progress_apply(rating_to_sentiment)

100%|██████████| 20491/20491 [00:00<00:00, 436659.38it/s]


In [42]:
from torchtext.data.functional import to_map_style_dataset
train_iter = [(label,text)for label,text in zip(csv['Sentiment'].to_list(),csv['Review'].to_list())]
train_dataset = to_map_style_dataset(train_iter)

In [43]:
from torch.utils.data.dataset import random_split
num_train = int(len(train_dataset) * 0.95)
split_train_dataset, split_test_dataset = random_split(train_dataset, [num_train, len(train_dataset) - num_train])


In [44]:
# Function to tokenize the text
def yield_tokens(data_iter,tokenizer):
    for _, text in data_iter:
        yield tokenizer(text)

# Build vocabulary from tokens of training set
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(yield_tokens(train_iter,tokenizer), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define collate_batch function to get single collated tensor for batch in form needed by nn.EmbeddingBag
def collate_batch(batch,tokenizer,vocab):
    # Pipelines for processing text and labels
    text_pipeline = lambda x: vocab(tokenizer(x))
    label_pipeline = lambda x: int(x) - 1
    
    label_list, text_list, offsets = [], [], [0]
    # Iterate through batch, processing text and adding text, labels and offsets to lists
    for (label, text) in batch:
        label_list.append(label_pipeline(label))
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)  

In [46]:
batch_size = 32
# Create training, validation and test set DataLoaders using custom collate_batch function
train_dataloader = DataLoader(split_train_dataset, batch_size=batch_size,
                              shuffle=True, collate_fn = lambda x: collate_batch(x,tokenizer,vocab))
val_dataloader = DataLoader(split_test_dataset, batch_size=batch_size,
                              shuffle=True, collate_fn = lambda x: collate_batch(x,tokenizer,vocab))



In [47]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(LSTMModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=True)
   
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, num_classes)

        self.init_weights()
    
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets).unsqueeze(1)
        lstm_output, (ht, ct) = self.lstm(embedded)
        #print(embedded.shape, self.embedding(text, offsets).shape,ht.shape,ht[-1].shape,self.fc(ht[-1]).shape)
        return self.fc(ht[-1])

# Model parameters
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 64
num_classes = 3

# Create the LSTM model, loss function, and optimizer
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    model.train()

    for (labels, text, offsets) in tqdm(train_dataloader):
        text = text.to(device)
        labels = labels.to(device)
        offsets = offsets.to(device)
       # print('text.shape:',text.shape)

        optimizer.zero_grad()
        outputs = model(text, offsets)
        
        loss = criterion(outputs, labels)
   
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()
        

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataset)

 

    print(f'Epoch {epoch+1}/{num_epochs}, train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}')


100%|██████████| 609/609 [00:05<00:00, 103.93it/s]


Epoch 1/100, train_loss: 0.6750, train_acc: 0.7144


100%|██████████| 609/609 [00:05<00:00, 121.44it/s]


Epoch 2/100, train_loss: 0.5049, train_acc: 0.7732


100%|██████████| 609/609 [00:04<00:00, 128.53it/s]


...

Epoch 98/100, train_loss: 0.0501, train_acc: 0.9419


100%|██████████| 609/609 [00:05<00:00, 117.90it/s]


Epoch 99/100, train_loss: 0.0265, train_acc: 0.9454


100%|██████████| 609/609 [00:05<00:00, 117.26it/s]

Epoch 100/100, train_loss: 0.0527, train_acc: 0.9380


In [48]:
import torch

# Assuming `model` is your trained PyTorch model
torch.save(model.state_dict(), 'model_LSTM.pth')


In [51]:
loaded_model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_classes)
loaded_model.load_state_dict(torch.load('model_LSTM.pth'))
loaded_model.eval()


LSTMModel(
  (embedding): EmbeddingBag(64280, 128, mode='mean')
  (lstm): LSTM(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=3, bias=True)
)

In [79]:
import torch
from sklearn.metrics import classification_report

def evaluate(dataloader, model):
    model.eval()
    all_labels, all_predictions = [], []

    with torch.no_grad():
        for idx, (label, text, offset) in enumerate(dataloader):
            predict = model(text, offset)
            predicted_labels = predict.argmax(1)
            all_labels.extend(label.tolist())
            all_predictions.extend(predicted_labels.tolist())

    report = classification_report(all_labels, all_predictions)
    return report
accu_test = evaluate(val_dataloader,model)
print(accu_test)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       145
           1       0.97      0.88      0.92       128
           2       0.98      1.00      0.99       752

    accuracy                           0.98      1025
   macro avg       0.98      0.95      0.96      1025
weighted avg       0.98      0.98      0.98      1025

